In [1]:
!pip install jupyter_dash 

     |████████████████████████████████| 7.4 MB 35.4 MB/s 
     |████████████████████████████████| 357 kB 45.6 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3822 sha256=4cf2e8a8afac52d92552f8a646ba85b1467bf66ccdc8158ee0c5cd5452aebd77
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4090 sha256=0199c410b71e25517bc39c9b83f473a39ddc06177fd11492c80a31d8900d28e2
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3912 sha256=c892279da58bb52e51f6e4343fd4fae694fbccc7621a0fc7ac4b86232b4886e3
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aefd2e2
  Created wheel for retrying: filename=retrying-1.3.

In [2]:
! pip install -q jupyter_dash==0.3.0
! pip install -q dash==1.19.0

     |████████████████████████████████| 49 kB 3.0 MB/s 
     |████████████████████████████████| 75 kB 3.2 MB/s 
     |████████████████████████████████| 1.0 MB 42.6 MB/s 
     |████████████████████████████████| 3.5 MB 42.5 MB/s 
     |████████████████████████████████| 188 kB 56.6 MB/s 
     |████████████████████████████████| 1.8 MB 46.0 MB/s 


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

1. To Do: use your data base and function to calculate the seven day average from previous exercise.

Creating a Dash Layout

In [4]:
app = JupyterDash(__name__) ## Jupyter Dash app

2. To Do: Create a function that returns a bar plot using graph_objects. 

In [5]:
#TODO
def movingaverage(interval, window_size=7):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [10]:
def bar(data, labels):
  return go.Bar(name="Raw data",x=labels, y=data)

3. To Do: Create Create a function that returns a line plot using graph_objects.

In [11]:
#TODO
def line(data,labels):
  return go.Scatter(name="7 Day moving avg",x=labels, y=data)

We can create a Figure that contains the plots:

In [12]:
 # Function that creates a Plotly Figure. 
 # The Figure displays the plots contained in 'input'.
 # The figure will have a width of 'w' and a height of 'h'
def create_fig(input, w = 900, h = 600 ):
  fig = go.Figure(data = input).update_layout(
                  width=w,  # setting  width
                  height=h, # setting  height   
                  updatemenus = list([dict(
                                type="buttons", # Buttons to activate/deactivate plots
                                active=0,       # First button will be active by default                 
                                buttons=list([
                                            # Button 1
                                            dict(label = 'Raw daily cases', # name of the button
                                                    method = 'update',        # method used to refresh the image
                                                    args = [{'visible': [True, False]}, # which plot will be available (only first) 
                                                            {'title': 'Covid-19 Cases in Germany'}]), # name of the plot
                                            # Button 2
                                            dict(label = '7 Day average', # name of the button
                                                    method = 'update',
                                                    args = [{'visible': [False, True]},
                                                            {'title': 'Covid-19 Cases in Germany'}]), 
                                            # Button 3
                                            dict(label = 'Both',
                                                    method = 'update',
                                                    args = [{'visible': [True, True]},
                                                            {'title': 'Covid-19 Cases in Germany'}])
                            ]),
                        )
                    ])
                )
  return fig


In [23]:
data = pd.read_csv('https://data.covid19bharat.org/csv/latest/case_time_series.csv')
date = np.array(data['Date_YMD'])
dailyCases = data['Daily Confirmed']
totalCases = data['Total Confirmed']
totalRecovered = data['Total Recovered']
totalDeceased = data['Total Deceased']
i= np.where(date == '2021-01-01')[0][0]
date_New = [d.date() for d in pd.to_datetime(date[i :])]
dailyCasesNew = dailyCases[i : -15]
dailyCasesNew = dailyCasesNew.astype(np.float)
weekly_average = movingaverage(dailyCasesNew)
totalCasesNew = totalCases[i :-15]
totalCasesNew = totalCasesNew.astype(np.float)
totalRecoveredNew = totalRecovered[i :-15]
totalRecoveredNew = totalRecoveredNew.astype(np.float)
totalDeceasedNew = totalDeceased[i :-15]
totalDeceasedNew = totalDeceasedNew.astype(np.float)

In [28]:
input = [bar(dailyCasesNew,date_New), line(weekly_average,date_New)] #bar plot, line plot

fig = create_fig(input)

4. To Do: Create a Dropdown that will update the data of your plots. Name it ``'dd'``

In [29]:
# TODO
# dd = dropdown
dd = dcc.Dropdown(id = 'info',
   options=[
        {'label': 'Daily cases', 'value': 'Daily Confirmed'},
       {'label': 'Total Cases', 'value': 'Total Confirmed'},
       {'label': 'Total Recovered', 'value': 'Total Recovered'},
       {'label': 'Total Deceased', 'value': 'Total Deceased'},
   ]
)


We create the final layout with a the ``Content Division`` element of the html:

In [30]:
app.layout = html.Div([
    html.H1("Covid cases in ??? "),   # Title
    dcc.Graph(id='graph', figure = fig),  # Graph that contains the figure (Look at the ID!!) 
    dd, # Dropdown under the figure
])

Working with callbacks

5. To Do: Create the ``callback`` to update your plot. Calcualte again the seven day average and make the plots using the new entry. Then, feed the graph with your new figure.

In [31]:
# TODO 
@app.callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='info', component_property='value')
)

def update_graph(data_required):
  date = np.array(data['Date_YMD'])
  newData = data[data_required]
  i= np.where(date == '2021-01-01')[0][0]
  dateNew = [d.date() for d in pd.to_datetime(date[i :])]

  totalNew = newData[i :-15].astype(np.float)

  avgnew = movingaverage(totalNew)

  bar = bar(totalNew,dateNew)
  line = line(avgnew,dateNew)
  in_val = [bar, line] #bar plot, line plot
  fig = create_fig(in_val)
  fig.update_layout(transition_duration = 500)
  
  return fig

Finally, we run the app using the port 8090. You can either follow the link, or use the ``inline mode`` to display the app on a cell of Jupyter Notebook/ Google Colab.

In [32]:
if __name__ == '__main__':
  app.run_server(port = 8090, dev_tools_ui=True, #mode='inline', #debug=True,
              dev_tools_hot_reload =True, threaded=True)

Dash app running on:


<IPython.core.display.Javascript object>